In [1]:
## INPUT
name="RiseOfTiamat"

In [2]:
import pandas as pd
import ast
import datetime
import re
from bs4 import BeautifulSoup

skill_labels=['ACROBATICS',
 'ANIMAL HANDLING',
 'ARCANA',
 'ATHLETICS',
 'DECEPTION',
 'HISTORY',
 'INSIGHT',
 'INTIMIDATION',
 'INVESTIGATION',
 'MEDICINE',
 'NATURE',
 'PERCEPTION',
 'PERFORMANCE',
 'PERSUASION',
 'RELIGION',
 'SLEIGHT OF HAND',
 'STEALTH',
 'SURVIVAL']

attr_labels=['STRENGTH',
 'DEXTERITY',
 'CONSTITUTION',
 'INTELLIGENCE',
 'WISDOM',
 'CHARISMA']

save_labels=['STRENGTH SAVE',
 'DEXTERITY SAVE',
 'CONSTITUTION SAVE',
 'INTELLIGENCE SAVE',
 'WISDOM SAVE',
 'CHARISMA SAVE',
 'DEATH SAVE']

# read player names and get all player names
file = open("players/players_"+name+".txt", "r")
contents = file.read()
players = ast.literal_eval(contents)
file.close()
plyrs=list(set(players.values()))

print("Loading chat log...")
soup = BeautifulSoup(open('raw/'+name+'.html', 'r').read(), 'html.parser')
msgs=soup.findAll('div',{'class' : re.compile('message \w+')})

print("Parsing messages...")
plyr=None
when=None
df = pd.DataFrame(columns=['Player','Timestamp','Skill/Weapon/Spell','Result', 'Raw','Plus'])
for c,m in enumerate(msgs):
    if c % 500 == 0:
        print(c+1,"/",len(msgs))
    
    ### get player and timestamp
    who=m.findAll('span',{'class': 'by'})
    if who!=[]:
        plyr=who[0].renderContents()
        when=m.find('span',{'class': 'tstamp'}).renderContents()
    

    log=True
    parsing=None
    skchk=m.findAll('div',{'class': ['sheet-rolltemplate-skill','sheet-rolltemplate-simple']})
    atk=m.findAll('div',{'class': ['sheet-rolltemplate-atk']})
    if skchk!=[] and atk!=[]:
        print("Figure out this weirdness! Skill and attack!")
    elif skchk!=[] and atk==[]:
        parsing="Skill"
        mcontent=skchk
    elif atk!=[] and skchk==[]:
        parsing="Attack"
        mcontent=atk

    if parsing!=None:   
        rolls=mcontent[0].find_all('div',{'class': ['sheet-roll','sheet-adv','sheet-solo']})
        if rolls==[]:
            log=False
        elif len(rolls)==1:
            roll=rolls[0]
        elif len(rolls)==2:
            roll=rolls[0] if rolls[1].find('span',{'class': 'sheet-grey'})!=None else rolls[1]
        else:
            log=False
        result=roll.find('span',{'class': lambda x: x and 'inlinerollresult' in x}).renderContents()

        tmplabel=mcontent[0].find('div',{'class': 'sheet-label'}).renderContents()
        if parsing=="Skill":
            tmplabel=re.sub(r'[^ A-Z0123456789-]','',str(tmplabel)).split(' ')
            label= [" ".join(tmplabel[:-1]),tmplabel[-1]] if len(tmplabel)>2 else tmplabel
        else:
            label=[re.sub("</*[a-zA-Z]+","",str(tmplabel).split(">")[2]),
                   re.sub("[^0-9-+]","",str(tmplabel).split(">")[4])] if rolls!=[] else ["",""]
            label[1]=label[1] if label[1]!='' else "0"


        if  log and (label[0] not in ["F","D K","T T","INITIATIVE","","D","S","C","P","H K","P S","F K","(+3)"]) and ("HIT DICE" not in label[0]) and (label[0] not in attr_labels):
            df.loc[str(c), 'Player'] = re.sub(':','',plyr.decode('UTF-8'))
            df.loc[str(c), 'Timestamp'] = when.decode('UTF-8')
            df.loc[str(c), 'Skill/Weapon/Spell'] = str(label[0])
            df.loc[str(c), 'Result'] = result.decode('UTF-8')
            df.loc[str(c), 'Raw'] = str(int(float(result))-int(label[1]))
            df.loc[str(c), 'Plus'] = str(label[1])
            df.loc[str(c), 'Final'] = 1
            df.loc[str(c), 'Type'] = parsing
            if len(rolls)==2:
                rollgrey=rolls[1] if rolls[1].find('span',{'class': 'sheet-grey'})!=None else rolls[0]
                resultgrey=rollgrey.find('span',{'class': lambda x: x and 'inlinerollresult' in x}).renderContents()
                df.loc[str(c)+".2", 'Player'] = re.sub(':','',plyr.decode('UTF-8'))
                df.loc[str(c)+".2", 'Timestamp'] = when.decode('UTF-8')
                df.loc[str(c)+".2", 'Skill/Weapon/Spell'] = str(label[0])
                df.loc[str(c)+".2", 'Result'] = resultgrey.decode('UTF-8')
                df.loc[str(c)+".2", 'Raw'] = str(int(float(resultgrey))-int(label[1]))
                df.loc[str(c)+".2", 'Plus'] = str(label[1])
                df.loc[str(c)+".2", 'Final'] = 0
                df.loc[str(c)+".2", 'Type'] = parsing
        
df["Skill/Weapon/Spell"]=df["Skill/Weapon/Spell"].str.strip()
df["Timestamp"]=pd.to_datetime(df["Timestamp"])
df.loc[df.Timestamp.dt.hour<6,"Timestamp"] = df["Timestamp"] + datetime.timedelta(days=1)
df["Player"]=df["Player"].apply(lambda x: players[x])
df['Day'] = df['Timestamp'].dt.strftime('%d.%m.%Y')
df.loc[(df["Type"]=="Skill")&(df["Skill/Weapon/Spell"].isin(skill_labels)==False)&(df["Skill/Weapon/Spell"].str.contains("SAVE")),"Type"]="Save"
df.loc[(df["Type"]=="Skill")&(df["Skill/Weapon/Spell"].isin(skill_labels)==False)&(df["Skill/Weapon/Spell"].str.contains("SAVE")==False),"Type"]="Other"
df.to_csv("rolls_"+name+".csv",index=False,sep=";")
df.head()

Loading chat log...
Parsing messages...
1 / 5400
501 / 5400
1001 / 5400
1501 / 5400
2001 / 5400
2501 / 5400
3001 / 5400
3501 / 5400
4001 / 5400
4501 / 5400
5001 / 5400


,Player,Timestamp,Skill/Weapon/Spell,Result,Raw,Plus,Final,Type,Day
10,Gruumsh,2020-04-19 19:09:00,PERSUASION,24,18,6,1.0,Skill,19.04.2020
10.2,Gruumsh,2020-04-19 19:09:00,PERSUASION,23,17,6,0.0,Skill,19.04.2020
16,Gruumsh,2020-04-19 20:24:00,INVESTIGATION,10,11,-1,1.0,Skill,19.04.2020
16.2,Gruumsh,2020-04-19 20:24:00,INVESTIGATION,0,1,-1,0.0,Skill,19.04.2020
17,Ylva,2020-04-19 20:34:00,PERCEPTION,25,19,6,1.0,Skill,19.04.2020


In [3]:
df[df["Type"]=="Other"]

,Player,Timestamp,Skill/Weapon/Spell,Result,Raw,Plus,Final,Type,Day
